# Setting up the environment and importing libraries

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

#dotenv files: https://dev.to/jakewitcher/using-env-files-for-environment-variables-in-python-applications-55a1

openai_key = os.getenv("OPENAI_API_KEY")

os.environ["OPENAI_API_KEY"] = openai_key

In [ ]:
import os
import sys

sys.path.insert(1, "/Users/atharvaparikh/Desktop/Purdue/ChopraLab/") 

from langchain_openai import ChatOpenAI
from sciborg.ai.agents.core import create_linqx_chat_agent
from sciborg.ai.chains.microservice import module_to_microservice
from sciborg.ai.chains.workflow import create_workflow_planner_chain, create_workflow_constructor_chain

from sciborg.testing.models.drivers import MicrowaveSynthesizer, MicrowaveSynthesizerObject, PubChemCaller
from sciborg.core.library.base import BaseDriverMicroservice

In [ ]:
file_path = '/Users/atharvaparikh/Desktop/Purdue/ChopraLab/sciborg/ai/agents/driver_pubchem.json'

pubchem_command_microservice = module_to_microservice(PubChemCaller)
print(pubchem_command_microservice)

with open(file_path, 'w') as outfile:
    outfile.write(pubchem_command_microservice.model_dump_json(indent=2))

pubchem_driver_based_agent = create_linqx_chat_agent(
    microservice=pubchem_command_microservice,
    llm=ChatOpenAI(model='gpt-4'),
    human_interaction=True,
    verbose=True
)

{'query': '\n    Create a command from the below information. \n    Do not include any parameters that are not in the function signature.\n    If the function signature denotes that it returns something, check to docstring to find the return signature.\n\n    Function name: get_assay_description\n    Microservice: PubChemCaller\n    UUID: fe254323-69eb-466f-b98f-0da3bde5999b\n    Function Signature: (aid: str) -> dict\n    Docstring: Function purpose\nGet high level Assay description, protocol and comment on the scores for a given assay id.\n\nInputs\naid: string representation of a list of assay IDs\n\nreturns\ndict: a dictionary of description, protocol and comment for the given assay ID\n    ', 'text': {'name': 'get_assay_description', 'microservice': 'PubChemCaller', 'desc': 'A short description of the command', 'uuid': 'fe254323-69eb-466f-b98f-0da3bde5999b', 'parameters': {'aid': {'name': 'aid', 'data_type': 'str', 'is_optional': False, 'desc': 'A string representation of a list o

In [ ]:
file_path = '/Users/atharvaparikh/Desktop/Purdue/ChopraLab/sciborg/ai/agents/driver_MicroSynth.json'

driver_command_microservice = module_to_microservice(MicrowaveSynthesizer)
print(type(driver_command_microservice))

with open(file_path, 'w') as outfile:
    outfile.write(driver_command_microservice.model_dump_json(indent=2))

{'query': '\n    Create a command from the below information. \n    Do not include any parameters that are not in the function signature.\n    If the function signature denotes that it returns something, check to docstring to find the return signature.\n\n    Function name: allocate_session\n    Microservice: MicrowaveSynthesizer\n    UUID: 29107d1a-c7bc-4770-977d-badf3f1ac79c\n    Function Signature: () -> dict\n    Docstring: Allocates a session on the microwave synthesizer.\nMust be called prior to any other action.\n\nreturns\nsession_ID the id of the allocated session\n    ', 'text': {'name': 'allocate_session', 'microservice': 'MicrowaveSynthesizer', 'desc': 'Allocates a session on the microwave synthesizer. Must be called prior to any other action.', 'uuid': '29107d1a-c7bc-4770-977d-badf3f1ac79c', 'parameters': {}, 'has_return': True, 'return_signature': {'session_ID': 'the id of the allocated session'}}}
{'query': '\n    Create a command from the below information. \n    Do not

# RAG Agent

In [ ]:
from langchain_openai import ChatOpenAI
from sciborg.utils.benchmarking.output import AgentRegexOutputBenchmarker

executor_kwargs = {
    'microservice': driver_command_microservice,
    'rag_vectordb_path' : '/Users/atharvaparikh/Desktop/Purdue/ChopraLab/RAG-LINQX/NIH_Aco_only_doc_embeddings',
    'llm': ChatOpenAI(model='gpt-4'),
    'human_interaction': False,
    'agent_as_a_tool': None,
    'use_linqx_tools': True,
}

# The regex pattern to use as desired_output

desired_output = r'(?=.*\b[Pp]ressures?\s*:\s*1\s*(?:atm|bar)(?:\s*\(\s*1\s*atm\s*\))?\b)(?=.*\b[Tt]emperatures?\s*:\s*120\s*°?\s*[Cc]\b)(?=.*\b(?:[Dd]urations?|[Tt]imes?)\s*.*?\b(?:5\s*,\s*10\s*,\s*20\s*,\s*30\s*,\s*45\s*,?\s*and\s*60\s*minutes?|60\s*minutes)\b)'

regex_output_benchmark = AgentRegexOutputBenchmarker(
    executor_fn=create_linqx_chat_agent,
    executor_kwargs=executor_kwargs,
    initial_input="can you recommend paramaters for an n-alkylation reaction using 3-Bromoquinoline and morpholine [II] as reactants. Provide the response as one single continuous paragraph containing relevant information with the parameter values in exactly the following format: Pressure: value, Temperature: value, Time: list of values",
    verbose=True,
    notebook=True,
    desired_output=desired_output
)

In [30]:
regex_output_benchmark.benchmark(20)

  0%|          | 0/20 [00:00<?, ?it/s]

- Success on output key: output, matched a desired output
- Agent Output: For an n-alkylation reaction using 3-Bromoquinoline and morpholine as reactants, the recommended parameters are as follows: Pressure: 1 bar, Temperature: 120 °C. The reaction times tested were 5, 10, 20, 30, 45, and 60 minutes. These parameters were used in a Buchwald-Hartwig reaction screen using a Microwave Reactor (Biotage Initiator+). Please note that the ratio of the reactants used was 3-Bromoquinoline: 1.0 equivalent and Morpholine: 1.2 equivalents. The catalyst employed was PEPPSI (formulation - ChemBeads) and the solvents tested included Dioxane and Toluene.
- Iteration 1: Success
- Success: 1, Fail: 0, Total: 1
- Benchmarking Score: 1.0
--------------------
- Success on output key: output, matched a desired output
- Agent Output: The recommended parameters for an n-alkylation reaction using 3-Bromoquinoline and morpholine [II] as reactants are as follows: Pressure: 1 atm, Temperature: 120°C. The reaction

In [ ]:
from langchain_openai import ChatOpenAI
from sciborg.utils.benchmarking.output import AgentRegexOutputBenchmarker

executor_kwargs = {
    'microservice': driver_command_microservice,
    'rag_vectordb_path' : '/Users/atharvaparikh/Desktop/Purdue/ChopraLab/sciborg/embeddings/NIH_docs_embeddings',
    'llm': ChatOpenAI(model='gpt-4'),
    'human_interaction': False,
    'agent_as_a_tool': None,
    'use_linqx_tools': True,
}

# The regex pattern to use as desired_output
desired_output = r'(?=.*\b(?:[Mm]icrowave\s[Ii]rradiation|[Cc]onventional\s[Hh]eating\s[Mm]ethods?)\b)'
#TODO:

regex_output_benchmark = AgentRegexOutputBenchmarker(
    executor_fn=create_linqx_chat_agent,
    executor_kwargs=executor_kwargs,
    initial_input="How does microwave irradiation influence reaction mechanisms differently compared to conventional heating methods?",
    verbose=True,
    notebook=True,
    desired_output=desired_output
)

In [8]:
#TODO: can I somehow go up the definitions and then change the verbose flag to False (as default it is True in file rag_agent.py)

In [9]:
regex_output_benchmark.benchmark(20)

  0%|          | 0/20 [00:00<?, ?it/s]

- Success on output key: output, matched a desired output
- Agent Output: Microwave irradiation influences reaction mechanisms differently compared to conventional heating methods by providing several advantages. First, microwave irradiation is a well-established method for increasing the reaction rate of chemical processes by generating high temperatures and pressures under controlled conditions, allowing for quicker reactions. Second, unlike conventional heating methods where thermal gradients can lead to non-uniform transformations and obstacles for scale-up production, microwave irradiation provides uniform heating to the reaction materials in the entire reaction vessel, enabling rapid and uniform heat transfer. Third, microwave radiation of suitable wavelength collaborates with the reaction material and generates heat simultaneously, leading to efficient and rapid reactions. A comparison between microwave and conventional heating methods shows that microwave-assisted reactions hav

In [ ]:
from langchain_openai import ChatOpenAI
from sciborg.utils.benchmarking.output import AgentRegexOutputBenchmarker

executor_kwargs = {
    'microservice': driver_command_microservice,
    'rag_vectordb_path' : '/Users/atharvaparikh/Desktop/Purdue/ChopraLab/sciborg/embeddings/NIH_docs_embeddings',
    'llm': ChatOpenAI(model='gpt-4'),
    'human_interaction': False,
    'agent_as_a_tool': None,
    'use_linqx_tools': True,
}

# The regex pattern to use as desired_output
desired_output = r'(?=.*\b(?:[Pp]ressure\s[Ss]etting|[Mm]icrowave|[Ss]ynthesizer|[Bb]iotage\s[Ii]nitiator\+)\b)'

regex_output_benchmark = AgentRegexOutputBenchmarker(
    executor_fn=create_linqx_chat_agent,
    executor_kwargs=executor_kwargs,
    initial_input="What function does the pressure setting serve on the Biotage Initiator+?",
    verbose=True,
    notebook=True,
    desired_output=desired_output
)

In [11]:
regex_output_benchmark.benchmark(20)

  0%|          | 0/20 [00:00<?, ?it/s]

- Success on output key: output, matched a desired output
- Agent Output: The pressure setting on the Biotage Initiator+ allows for temperatures and pressures of up to 300 °C and 30 bar. This feature provides the ability to perform complex reactions that would be difficult under other conditions. Specifically, it allows solvents with low boiling points to be used at higher temperatures, providing more flexibility in choosing a solvent for organic synthesis.
- Iteration 1: Success
- Success: 1, Fail: 0, Total: 1
- Benchmarking Score: 1.0
--------------------
- Success on output key: output, matched a desired output
- Agent Output: The pressure setting on the Biotage Initiator+ allows for temperatures and pressures of up to 300 °C and 30 bar, which opens up new possibilities to perform difficult reactions. This feature enables even solvents with low boiling points to be run at higher temperatures, providing more flexibility in choosing a solvent for reactions.
- Iteration 2: Success
- Su